제안 2: 해약률 분석
문제 정의: 다양한 변수들(계약 유형, 납부 유형, 연령 등)을 분석하여 고객의 계약 해약률을 분석합니다.
배경: 해약률을 분석함으로써 위험 요소를 식별하고, 해약률을 감소시킬 수 있는 전략을 개발합니다.


In [119]:
import matplotlib.pyplot as plt
import matplotlib as mpl # 한글 폰트 설정 (NanumGothic) 
mpl.rcParams['font.family'] = 'NanumGothic'
mpl.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지
import seaborn as sns
import pandas as pd
import numpy as np 

In [120]:
data_df = pd.read_csv('../../datasets/LetalCarOfContractType.csv')
data_df.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0


In [121]:
data_df.info(), data_df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51304 entries, 0 to 51303
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 51304 non-null  int64  
 1   type_of_contract   51300 non-null  object 
 2   type_of_contract2  51303 non-null  object 
 3   channel            51304 non-null  object 
 4   datetime           51304 non-null  object 
 5   Term               51304 non-null  int64  
 6   payment_type       51304 non-null  object 
 7   product            51303 non-null  object 
 8   amount             51304 non-null  int64  
 9   state              51304 non-null  object 
 10  overdue_count      51304 non-null  int64  
 11  overdue            51302 non-null  object 
 12  credit rating      42521 non-null  float64
 13  bank               48544 non-null  object 
 14  cancellation       51279 non-null  object 
 15  age                40509 non-null  float64
 16  Mileage            405

(None,
                  id          Term         amount  overdue_count  \
 count  5.130400e+04  51304.000000   51304.000000   51304.000000   
 mean   6.266457e+07     55.638527   93994.980021       0.161040   
 std    1.216115e+07     12.010117   15305.147334       1.122356   
 min    2.568798e+07     12.000000   54603.000000      -1.000000   
 25%    6.643149e+07     60.000000   81900.000000       0.000000   
 50%    6.676578e+07     60.000000   96900.000000       0.000000   
 75%    6.678161e+07     60.000000   98400.000000       0.000000   
 max    6.696986e+07     60.000000  215700.000000      15.000000   
 
        credit rating           age       Mileage  
 count   42521.000000  40509.000000  40509.000000  
 mean        3.428894     46.608285   1967.922412  
 std         2.213590     13.465977    436.707329  
 min         0.000000     20.000000    872.000000  
 25%         1.000000     37.000000   1646.000000  
 50%         3.000000     46.000000   1946.000000  
 75%         5.

## 해약률 분포 
- state 가 cancellation 에 비해 nan 이 없고, 더 자세한 정보 담고 있으므로, state 사용
- 해약확정 과 해약진행중 에 대해서 진행 

In [122]:
data_df['state'].unique()

array(['계약확정', '해약확정', '기간만료', '해약진행중'], dtype=object)

In [123]:
data_df['state'].value_counts()

state
계약확정     50350
해약확정       907
기간만료        33
해약진행중       14
Name: count, dtype: int64

In [124]:
data_df['cancellation'].unique()

array(['정상', '해약', nan], dtype=object)

In [125]:
data_df['cancellation'].value_counts()

cancellation
정상    50372
해약      907
Name: count, dtype: int64

#### 조건 정리
1. 나이 : 40대가 199건 1.8% 로 해약 비율과 해약건이 가장 많았다.
2. 채널 : 렌탈재계약 일 때 18.9%의 높은 해약률을 보였다. 
3. 계약 : 멤버십의 경우에 해약율이 높았으며, 특히, 멤버십_TA3 12%, 멤버십_TAS 18.9% 의 해약율을 보였다. 
4. 신용 : 전 구간에서 비슷한 해약율을 보인것으로 보아 상관 관계가 없어 보인다.
5. 연체 : 연체 상태와는 큰 연관성이 없는 것으로 보인다. 
6. 납부유형 : 카드이체의 경우 전체 계약의 30%의 많은 계약을 체결 했는데 해약율이 0.7%의 낮은 해약율을 보였다.  
7. 마일리지 : 전 구간에서 비슷한 해약율을 보인것으로 보아 상관 관계가 없어 보인다.
8. 금액 : 금액이 적을 수록 해약율이 높았다. 4.6%
9. 계약기간 : 주로 12개월 계약일 때 해약율이 19.2% 로 가장 높았다.

인싸이트 종합: 나이가 40대 일 때, 채널이 렌탈재계약 일 때, 계약이 멤버십 일 때, 납부유형이 카드이체가 아닐 때, 금액이 적을 때, 기간이 12개월 일 때, 해약율이 높았다. 


#### 해결방안 
1. 위 구간에 해당하는 타겟 구간에 들어가게 되면 해약 위험 관리 군으로 지정하여 혜택을 지원해야 할 것으로 보인다. 
2. 특히, 렌탈 재계약인 경우에 해약이 되는 것으로 보아 지금의 재계약 방식에 대한 재검토가 필요할 것으로 보인다.
3. 멤버십의 경우에도 해약율이 높은것으로 보아 현재의 멤버십 방식에 대한 재검토가 필요할 것으로 보인다. 
4. 따라서, 금액을 더 높은 상품에 혜택을 주던가, 카드이체에 혜택을 주던가, 기간이 12개월을 넘도록 유도 할 수 있는 방안이 필요할 것으로 보인다. 

### 방향 설정
- 우선 해약확정 과 해약진행중 인 건들에 대해서 보기
- 해약 하는 사람 들의 나이, 어떤 채널을 통해 들어왔는지 등을 확인해보기
- 해약률 = (해약계약건수 / 전체계약건수) * 100
  1. 나이 비교
   - 어떤 나이대의 사람들이 해약을 많이 하는지 
   - 인사이트 : 40대가 199건 1.8% 로 해약 비율과 해약건이 가장 많았다.

In [126]:
condition_state = 'state == "해약확정" or state == "해약진행중"'

In [127]:
# 연령별 CMS 분포 확인 
bins = [20,30,40,50,60]
labels = ['20-29','30-39','40-49','50-59']

data_df['age_group'] = pd.cut(data_df['age'], bins=bins, labels=labels)
data_df.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage,age_group
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0,40-49
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0,NaN
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0,50-59
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0,50-59
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0,50-59


In [128]:
cancell_df = data_df.query(condition_state)
cancell_df.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage,age_group
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0,50-59
44,66423714,멤버십,TAS,렌탈재계약,2019-10-21,12,CMS,K1,66900,해약확정,0,없음,3.0,국민은행,해약,31.0,1319.0,30-39
90,66879582,렌탈,Promotion,홈플러스,2019-10-21,60,CMS,K1,96900,해약확정,0,없음,5.0,신한은행,해약,46.0,1950.0,40-49
114,66423309,멤버십,TAS,렌탈재계약,2019-10-21,12,CMS,K1,78900,해약확정,0,없음,3.0,신한은행,해약,38.0,1669.0,30-39
167,66173172,멤버십,TAS,렌탈재계약,2019-10-21,12,CMS,K1,66900,해약확정,0,없음,NaN,농협은행,해약,NaN,NaN,NaN


In [129]:
cancell_df['age_group'].value_counts()

age_group
40-49    199
30-39    161
50-59    136
20-29     72
Name: count, dtype: int64

In [130]:
cancellation_rate = (cancell_df['age_group'].value_counts() / 
                    data_df['age_group'].value_counts() * 100).sort_index()
cancellation_rate

age_group
20-29    1.492228
30-39    1.738285
40-49    1.803843
50-59    1.541601
Name: count, dtype: float64

2. 어떤 채널을 통해 렌탈을 진행 했는지 
   - 인사이트 : 렌탈재계약 일 때 18.9%의 높은 해약률을 보였다. 

In [131]:
data_df['channel'].value_counts()

channel
서비스 방문     15186
홈쇼핑/방송     12977
렌탈총판        5544
R관리방판       4727
하이마트        3227
영업방판        3039
렌탈재계약       2193
일반          1232
전자랜드        1205
R빌리미         560
이마트          265
홈플러스         225
R인터넷         198
R유통사         188
홈쇼핑/인터넷      177
R렌탈운영        120
R농협          119
R농협인터넷몰       83
R법인그룹         20
R법인           19
Name: count, dtype: int64

In [132]:
cancell_df['channel'].value_counts()

channel
렌탈재계약      414
R관리방판      132
서비스 방문     111
하이마트        76
홈쇼핑/방송      52
렌탈총판        49
영업방판        36
R빌리미        18
전자랜드        12
일반           8
홈플러스         6
이마트          4
홈쇼핑/인터넷      2
R인터넷         1
Name: count, dtype: int64

In [133]:
cancellation_rate = (cancell_df['channel'].value_counts() / 
                    data_df['channel'].value_counts() * 100).sort_index()
cancellation_rate

channel
R관리방판       2.792469
R농협              NaN
R농협인터넷몰          NaN
R렌탈운영            NaN
R법인              NaN
R법인그룹            NaN
R빌리미        3.214286
R유통사             NaN
R인터넷        0.505051
렌탈재계약      18.878249
렌탈총판        0.883838
서비스 방문      0.730936
영업방판        1.184600
이마트         1.509434
일반          0.649351
전자랜드        0.995851
하이마트        2.355129
홈쇼핑/방송      0.400709
홈쇼핑/인터넷     1.129944
홈플러스        2.666667
Name: count, dtype: float64

3. 어떤 계약 유형의 렌탈을 진행 했는지 
   - 인사이트 : 멤버십의 경우에 해약율이 높았으며, 특히, 멤버십_TA3 12%, 멤버십_TAS 18.9% 의 해약율을 보였다.

In [134]:
# 1. 문자열 결합 방식
data_df['contract_type_combined'] = data_df['type_of_contract'] + '_' + data_df['type_of_contract2']
cancell_df = data_df.query(condition_state)

In [135]:
data_df['contract_type_combined'].value_counts()

contract_type_combined
렌탈_Promotion           15810
렌탈_Normal              15322
렌탈_Package             10184
렌탈_Extension_Rental     4974
멤버십_TAS                 2627
멤버십_TA2                 2087
렌탈_Group                 190
멤버십_TA3                   92
멤버십_TA1                   13
Name: count, dtype: int64

In [136]:
cancell_df['contract_type_combined'].value_counts()

contract_type_combined
멤버십_TAS                497
렌탈_Normal              146
렌탈_Promotion           133
멤버십_TA2                 55
렌탈_Package              54
렌탈_Extension_Rental     25
멤버십_TA3                 11
Name: count, dtype: int64

In [137]:
cancellation_rate = (cancell_df['contract_type_combined'].value_counts() / 
                    data_df['contract_type_combined'].value_counts() * 100).sort_index()
cancellation_rate

contract_type_combined
렌탈_Extension_Rental     0.502614
렌탈_Group                     NaN
렌탈_Normal               0.952878
렌탈_Package              0.530244
렌탈_Promotion            0.841240
멤버십_TA1                      NaN
멤버십_TA2                 2.635362
멤버십_TA3                11.956522
멤버십_TAS                18.918919
Name: count, dtype: float64

4. 신용상태와 연관이 있는지  
  -  인사이트 : 전 구간에서 비슷한 해약율을 보인것으로 보아 상관 관계가 없어 보인다.

In [138]:
data_df['credit_group'] = pd.cut(data_df['credit rating'], bins=5)
data_df.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,overdue_count,overdue,credit rating,bank,cancellation,age,Mileage,age_group,contract_type_combined,credit_group
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,9.0,새마을금고,정상,43.0,1862.0,40-49,렌탈_Normal,"(8.0, 10.0]"
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,0,없음,2.0,현대카드,정상,62.0,2532.0,NaN,렌탈_Extension_Rental,"(-0.01, 2.0]"
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,0,없음,8.0,우리은행,정상,60.0,2363.0,50-59,렌탈_Normal,"(6.0, 8.0]"
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,0,없음,5.0,농협은행,정상,60.0,2449.0,50-59,멤버십_TAS,"(4.0, 6.0]"
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,12,있음,8.0,농협은행,해약,51.0,1942.0,50-59,멤버십_TAS,"(6.0, 8.0]"


In [139]:
cancell_df = data_df.query(condition_state)
cancell_df['credit_group'].value_counts()

credit_group
(2.0, 4.0]      256
(-0.01, 2.0]    230
(4.0, 6.0]      134
(6.0, 8.0]       82
(8.0, 10.0]      14
Name: count, dtype: int64

In [140]:
data_df['credit_group'].value_counts()

credit_group
(-0.01, 2.0]    19066
(2.0, 4.0]      10376
(4.0, 6.0]       8135
(6.0, 8.0]       4259
(8.0, 10.0]       685
Name: count, dtype: int64

In [141]:
cancellation_rate = (cancell_df['credit_group'].value_counts() / 
                    data_df['credit_group'].value_counts() * 100).sort_index()
cancellation_rate

credit_group
(-0.01, 2.0]    1.206336
(2.0, 4.0]      2.467232
(4.0, 6.0]      1.647203
(6.0, 8.0]      1.925335
(8.0, 10.0]     2.043796
Name: count, dtype: float64

5. 연체상태와 연관이 있는지  
   -  인사이트 : 연체 상태와는 큰 연관성이 없는 것으로 보인다.   

In [142]:
cancell_df['overdue'].value_counts()

overdue
없음    883
있음     38
Name: count, dtype: int64

In [143]:
data_df['overdue'].value_counts()

overdue
없음    49112
있음     2190
Name: count, dtype: int64

In [144]:
cancellation_rate = (cancell_df['overdue'].value_counts() / 
                    data_df['overdue'].value_counts() * 100).sort_index()
cancellation_rate

overdue
없음    1.797931
있음    1.735160
Name: count, dtype: float64

In [145]:
data_df['overdue_group'] = pd.cut(data_df['overdue_count'], bins=5)
data_df.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,...,overdue,credit rating,bank,cancellation,age,Mileage,age_group,contract_type_combined,credit_group,overdue_group
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,...,없음,9.0,새마을금고,정상,43.0,1862.0,40-49,렌탈_Normal,"(8.0, 10.0]","(-1.016, 2.2]"
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,...,없음,2.0,현대카드,정상,62.0,2532.0,NaN,렌탈_Extension_Rental,"(-0.01, 2.0]","(-1.016, 2.2]"
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,...,없음,8.0,우리은행,정상,60.0,2363.0,50-59,렌탈_Normal,"(6.0, 8.0]","(-1.016, 2.2]"
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,...,없음,5.0,농협은행,정상,60.0,2449.0,50-59,멤버십_TAS,"(4.0, 6.0]","(-1.016, 2.2]"
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,...,있음,8.0,농협은행,해약,51.0,1942.0,50-59,멤버십_TAS,"(6.0, 8.0]","(11.8, 15.0]"


In [146]:
cancell_df = data_df.query(condition_state)
cancell_df['overdue_group'].value_counts()

overdue_group
(-1.016, 2.2]    904
(8.6, 11.8]        6
(11.8, 15.0]       5
(2.2, 5.4]         4
(5.4, 8.6]         2
Name: count, dtype: int64

In [147]:
data_df['overdue_group'].value_counts()

overdue_group
(-1.016, 2.2]    50474
(2.2, 5.4]         332
(11.8, 15.0]       213
(8.6, 11.8]        164
(5.4, 8.6]         121
Name: count, dtype: int64

In [148]:
cancellation_rate = (cancell_df['overdue_group'].value_counts() / 
                    data_df['overdue_group'].value_counts() * 100).sort_index()
cancellation_rate

overdue_group
(-1.016, 2.2]    1.791021
(2.2, 5.4]       1.204819
(5.4, 8.6]       1.652893
(8.6, 11.8]      3.658537
(11.8, 15.0]     2.347418
Name: count, dtype: float64

6. 납부유형과 연관이 있는지  
 -  인사이트 : 카드이체의 경우 전체 계약의 30%의 많은 계약을 체결 했는데 해약율이 0.7%의 낮은 해약율을 보였다.  


In [149]:
data_df['payment_type'].value_counts()

payment_type
CMS     32825
카드이체    15717
무통장      2085
가상계좌      612
지로         65
Name: count, dtype: int64

In [150]:
cancell_df['payment_type'].value_counts()

payment_type
CMS     757
카드이체    108
무통장      56
Name: count, dtype: int64

In [151]:
cancellation_rate = (cancell_df['payment_type'].value_counts() / 
                    data_df['payment_type'].value_counts() * 100).sort_index()
cancellation_rate

payment_type
CMS     2.306169
가상계좌         NaN
무통장     2.685851
지로           NaN
카드이체    0.687154
Name: count, dtype: float64

7. 마일리지와 연관이 있는지  
 -  인사이트 : 전 구간에서 비슷한 해약율을 보인것으로 보아 상관 관계가 없어 보인다.

In [152]:
data_df['Mileage_group'] = pd.cut(data_df['Mileage'], bins=5)
data_df.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,...,credit rating,bank,cancellation,age,Mileage,age_group,contract_type_combined,credit_group,overdue_group,Mileage_group
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,...,9.0,새마을금고,정상,43.0,1862.0,40-49,렌탈_Normal,"(8.0, 10.0]","(-1.016, 2.2]","(1458.2, 2044.4]"
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,...,2.0,현대카드,정상,62.0,2532.0,NaN,렌탈_Extension_Rental,"(-0.01, 2.0]","(-1.016, 2.2]","(2044.4, 2630.6]"
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,...,8.0,우리은행,정상,60.0,2363.0,50-59,렌탈_Normal,"(6.0, 8.0]","(-1.016, 2.2]","(2044.4, 2630.6]"
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,...,5.0,농협은행,정상,60.0,2449.0,50-59,멤버십_TAS,"(4.0, 6.0]","(-1.016, 2.2]","(2044.4, 2630.6]"
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,...,8.0,농협은행,해약,51.0,1942.0,50-59,멤버십_TAS,"(6.0, 8.0]","(11.8, 15.0]","(1458.2, 2044.4]"


In [153]:
data_df['Mileage_group'].value_counts()


Mileage_group
(1458.2, 2044.4]     18432
(2044.4, 2630.6]     13940
(869.069, 1458.2]     5185
(2630.6, 3216.8]      2797
(3216.8, 3803.0]       155
Name: count, dtype: int64

In [154]:
cancell_df = data_df.query(condition_state)
cancell_df['Mileage_group'].value_counts()


Mileage_group
(1458.2, 2044.4]     338
(2044.4, 2630.6]     219
(869.069, 1458.2]     73
(2630.6, 3216.8]      47
(3216.8, 3803.0]       3
Name: count, dtype: int64

In [155]:
cancellation_rate = (cancell_df['Mileage_group'].value_counts() / 
                    data_df['Mileage_group'].value_counts() * 100).sort_index()
cancellation_rate

Mileage_group
(869.069, 1458.2]    1.407907
(1458.2, 2044.4]     1.833767
(2044.4, 2630.6]     1.571019
(2630.6, 3216.8]     1.680372
(3216.8, 3803.0]     1.935484
Name: count, dtype: float64

8. 금액과 연관이 있는지  
 -  인사이트 : 금액이 적을 수록 해약율이 높았다. 4.6%

In [156]:
labels = ['1','2','3','4','5']
data_df['amount_group'] = pd.cut(data_df['amount'], bins=5, labels=labels)
data_df.head()

,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,...,bank,cancellation,age,Mileage,age_group,contract_type_combined,credit_group,overdue_group,Mileage_group,amount_group
0,66758234,렌탈,Normal,서비스 방문,2019-10-20,60,CMS,K1,96900,계약확정,...,새마을금고,정상,43.0,1862.0,40-49,렌탈_Normal,"(8.0, 10.0]","(-1.016, 2.2]","(1458.2, 2044.4]",2
1,66755948,렌탈,Extension_Rental,서비스 방문,2019-10-20,60,카드이체,K1,102900,계약확정,...,현대카드,정상,62.0,2532.0,NaN,렌탈_Extension_Rental,"(-0.01, 2.0]","(-1.016, 2.2]","(2044.4, 2630.6]",2
2,66756657,렌탈,Normal,홈쇼핑/방송,2019-10-20,60,CMS,K1,96900,계약확정,...,우리은행,정상,60.0,2363.0,50-59,렌탈_Normal,"(6.0, 8.0]","(-1.016, 2.2]","(2044.4, 2630.6]",2
3,66423450,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,계약확정,...,농협은행,정상,60.0,2449.0,50-59,멤버십_TAS,"(4.0, 6.0]","(-1.016, 2.2]","(2044.4, 2630.6]",1
4,66423204,멤버십,TAS,렌탈재계약,2019-10-20,12,CMS,K1,66900,해약확정,...,농협은행,해약,51.0,1942.0,50-59,멤버십_TAS,"(6.0, 8.0]","(11.8, 15.0]","(1458.2, 2044.4]",1


In [157]:
data_df['amount_group'].value_counts()


amount_group
2    35865
1    13058
3     2241
5      120
4       20
Name: count, dtype: int64

In [158]:
cancell_df = data_df.query(condition_state)
cancell_df['amount_group'].value_counts()

amount_group
1    595
2    311
3     13
5      2
4      0
Name: count, dtype: int64

In [159]:
cancellation_rate = (cancell_df['amount_group'].value_counts() / 
                    data_df['amount_group'].value_counts() * 100).sort_index()
cancellation_rate

amount_group
1    4.556594
2    0.867141
3    0.580098
4    0.000000
5    1.666667
Name: count, dtype: float64

9. 계약 기간과 연관이 있는지  
   -  인사이트 : 주로 12개월 계약일 때 해약율이 19.2% 로 가장 높았다.

In [160]:
data_df['Term'].value_counts()


Term
60    44536
36     3352
12     2651
39      765
Name: count, dtype: int64

In [161]:
cancell_df['Term'].value_counts()


Term
12    508
60    347
36     60
39      6
Name: count, dtype: int64

In [162]:
cancellation_rate = (cancell_df['Term'].value_counts() / 
                    data_df['Term'].value_counts() * 100).sort_index()
cancellation_rate

Term
12    19.162580
36     1.789976
39     0.784314
60     0.779145
Name: count, dtype: float64

종합: 나이가 40대 일 때, 채널이 렌탈재계약 일 때, 계약이 멤버십 일 때, 납부유형이 카드이체가 아닐 때, 금액이 적을 때, 기간이 12개월 일 때, 해약율이 높았다. 

In [163]:
condition_age = 'age_group == "40-49"'
condition_channel = 'channel == "렌탈재계약"'
condition_contract = 'type_of_contract == "멤버십"'
condition_term = 'Term == 12' 
condition_amount = 'amount_group == "1"'

In [164]:
cancell_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 921 entries, 4 to 51303
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   id                      921 non-null    int64   
 1   type_of_contract        921 non-null    object  
 2   type_of_contract2       921 non-null    object  
 3   channel                 921 non-null    object  
 4   datetime                921 non-null    object  
 5   Term                    921 non-null    int64   
 6   payment_type            921 non-null    object  
 7   product                 921 non-null    object  
 8   amount                  921 non-null    int64   
 9   state                   921 non-null    object  
 10  overdue_count           921 non-null    int64   
 11  overdue                 921 non-null    object  
 12  credit rating           716 non-null    float64 
 13  bank                    865 non-null    object  
 14  cancellation            921 n

In [165]:
cancell_df = data_df.query(condition_state)
# cancell_df.query(f'{condition_age} and {condition_channel} and {condition_contract} and {condition_term} and {condition_amount}')
total_df = cancell_df.query(f'{condition_age} and {condition_channel} and {condition_contract} and {condition_term} and {condition_amount}')

In [166]:
total_df


,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,...,bank,cancellation,age,Mileage,age_group,contract_type_combined,credit_group,overdue_group,Mileage_group,amount_group
519,66423671,멤버십,TAS,렌탈재계약,2019-10-21,12,CMS,K1,66900,해약확정,...,하나은행,해약,49.0,2006.0,40-49,멤버십_TAS,"(4.0, 6.0]","(-1.016, 2.2]","(1458.2, 2044.4]",1
774,66423977,멤버십,TAS,렌탈재계약,2019-10-22,12,CMS,K1,66900,해약확정,...,농협은행,해약,47.0,1826.0,40-49,멤버십_TAS,"(2.0, 4.0]","(-1.016, 2.2]","(1458.2, 2044.4]",1
2515,66424067,멤버십,TAS,렌탈재계약,2019-10-24,12,CMS,K2,55200,해약확정,...,국민은행,해약,44.0,1939.0,40-49,멤버십_TAS,"(2.0, 4.0]","(-1.016, 2.2]","(1458.2, 2044.4]",1
2585,66930260,멤버십,TAS,렌탈재계약,2019-10-24,12,CMS,K1,66900,해약확정,...,국민은행,해약,50.0,2415.0,40-49,멤버십_TAS,"(-0.01, 2.0]","(-1.016, 2.2]","(2044.4, 2630.6]",1
3686,66424506,멤버십,TAS,렌탈재계약,2019-10-26,12,CMS,K1,66900,해약확정,...,국민은행,해약,50.0,1995.0,40-49,멤버십_TAS,"(2.0, 4.0]","(-1.016, 2.2]","(1458.2, 2044.4]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36837,66437788,멤버십,TAS,렌탈재계약,2019-12-29,12,CMS,K1,66900,해약확정,...,농협은행,해약,47.0,2093.0,40-49,멤버십_TAS,"(2.0, 4.0]","(-1.016, 2.2]","(2044.4, 2630.6]",1
36930,66437589,멤버십,TAS,렌탈재계약,2019-12-29,12,CMS,K1,66900,해약확정,...,국민은행,해약,48.0,1879.0,40-49,멤버십_TAS,"(2.0, 4.0]","(-1.016, 2.2]","(1458.2, 2044.4]",1
43694,66475463,멤버십,TAS,렌탈재계약,2020-01-19,12,CMS,K2,55200,해약확정,...,새마을금고,해약,48.0,1899.0,40-49,멤버십_TAS,"(8.0, 10.0]","(-1.016, 2.2]","(1458.2, 2044.4]",1
49358,66441907,멤버십,TAS,렌탈재계약,2020-01-30,12,카드이체,K1,66900,해약확정,...,현대카드,해약,42.0,1812.0,40-49,멤버십_TAS,"(2.0, 4.0]","(-1.016, 2.2]","(1458.2, 2044.4]",1


In [167]:
cancell_df.query(f'not ({condition_channel}) and not ({condition_contract})')



,id,type_of_contract,type_of_contract2,channel,datetime,Term,payment_type,product,amount,state,...,bank,cancellation,age,Mileage,age_group,contract_type_combined,credit_group,overdue_group,Mileage_group,amount_group
90,66879582,렌탈,Promotion,홈플러스,2019-10-21,60,CMS,K1,96900,해약확정,...,신한은행,해약,46.0,1950.0,40-49,렌탈_Promotion,"(4.0, 6.0]","(-1.016, 2.2]","(1458.2, 2044.4]",2
190,66865779,렌탈,Promotion,일반,2019-10-21,60,CMS,K1,96900,해약확정,...,우체국,해약,NaN,NaN,NaN,렌탈_Promotion,NaN,"(-1.016, 2.2]",NaN,2
691,66953885,렌탈,Package,홈쇼핑/방송,2019-10-22,60,CMS,K1,81900,해약확정,...,농협은행,해약,NaN,NaN,NaN,렌탈_Package,NaN,"(-1.016, 2.2]",NaN,1
692,66953885,렌탈,Package,홈쇼핑/방송,2019-10-22,60,CMS,K2,81900,해약확정,...,농협은행,해약,NaN,NaN,NaN,렌탈_Package,NaN,"(-1.016, 2.2]",NaN,1
718,66878789,렌탈,Extension_Rental,서비스 방문,2019-10-22,60,CMS,K1,96900,해약확정,...,농협은행,해약,58.0,2296.0,50-59,렌탈_Extension_Rental,"(6.0, 8.0]","(-1.016, 2.2]","(2044.4, 2630.6]",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51295,66799197,렌탈,Promotion,영업방판,2020-02-03,39,카드이체,K5,120900,해약확정,...,롯데카드,해약,48.0,1869.0,40-49,렌탈_Promotion,"(-0.01, 2.0]","(-1.016, 2.2]","(1458.2, 2044.4]",3
51297,66799607,렌탈,Normal,홈쇼핑/방송,2020-02-03,60,CMS,K1,96900,해약확정,...,신한은행,해약,70.0,2623.0,NaN,렌탈_Normal,"(6.0, 8.0]","(-1.016, 2.2]","(2044.4, 2630.6]",2
51300,66799558,렌탈,Normal,영업방판,2020-02-03,60,카드이체,K1,96900,해약확정,...,롯데카드,해약,39.0,1753.0,30-39,렌탈_Normal,"(6.0, 8.0]","(-1.016, 2.2]","(1458.2, 2044.4]",2
51301,66799197,렌탈,Promotion,홈쇼핑/방송,2020-02-03,39,무통장,K5,120900,해약확정,...,NaN,해약,51.0,2217.0,50-59,렌탈_Promotion,"(-0.01, 2.0]","(-1.016, 2.2]","(2044.4, 2630.6]",3
